### Информация
- *Автор*: Киселев Кирилл
- *Дата*: вторник, 24 мая 2022 г.
- *Описание*: Блокнот для вычисления регионов для актов

#### Requirements

In [1]:
# !pip3 install opencv-python==4.4.0.46 pandas==1.2.4

In [2]:
import cv2
import pandas as pd


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
data = pd.read_csv('/home/kirillk/datasets/OCR/ocr_act_1/test-cases-act.csv', index_col=0)

# data.loc[data['filename'] == '095.jpg', 'orientation'] = 'horizontal'

data.head()
# data.to_csv('../assets/test-cases.csv')

,filename,doc_type,doc_number,doc_date,scan_is_good,doc_type_box_left,doc_type_box_top,doc_type_box_width,doc_type_box_height,doc_number_box_left,doc_number_box_top,doc_number_box_width,doc_number_box_height,doc_date_box_left,doc_date_box_top,doc_date_box_width,doc_date_box_height
0,001.jpg,акт,6017103091,24.11.2021,True,0.026561,0.014445,0.023020,0.008174,0.066498,0.013455,0.071328,0.010155,0.156660,0.013331,0.065398,0.009536
1,002.jpg,акт,6600174850,12.10.2021,True,0.037938,0.035595,0.023487,0.007534,0.079714,0.034500,0.072964,0.008492,0.172122,0.034500,0.065841,0.008766
2,003.jpg,акт,6600226583,07.12.2021,True,0.070513,0.037068,0.023613,0.008848,0.111159,0.036792,0.073161,0.008709,0.202126,0.036239,0.065419,0.009401
3,004.jpg,акт,6600236611,17.12.2021,True,0.033116,0.045036,0.023148,0.008422,0.073842,0.045036,0.071531,0.008422,0.164170,0.044789,0.065962,0.008669
4,005.jpg,акт,6600241100,22.12.2021,True,0.039245,0.047049,0.023848,0.007750,0.080277,0.047049,0.072069,0.007625,0.170582,0.046799,0.066808,0.008125


## Исследование по вычислениям регионов

### Расположение типа документа

Отступ

In [4]:
type_indent = data['doc_type_box_left'].min()
type_indent

0.0162229857887107

Ширина

In [5]:
data['doc_type_box_left'].max()

0.0841961177180831

In [6]:
data['doc_type_box_width'].max()

0.0269816594720191

In [7]:
doc_type_region = (data['doc_type_box_left'] + data['doc_type_box_width']).max()
doc_type_region

0.1076022241862487

In [8]:
doc_type_width = (data['doc_type_box_left'] + data['doc_type_box_width']).max() - data['doc_type_box_left'].min()
doc_type_width

0.09137923839753799

In [9]:
# doc_type_region = (data['doc_type_box_left'] + data['doc_type_box_width']).max()
# doc_type_region

#### Визуализация результатов

In [10]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_act_1/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.2, fy=0.2)
    height, width, _ = image.shape
    
    x1 = int(row['doc_type_box_left'] * width)
    y1 = int(row['doc_type_box_top'] * height)
    x2 = int((row['doc_type_box_left'] + row['doc_type_box_width']) * width)
    y2 = int((row['doc_type_box_top'] + row['doc_type_box_height']) * height)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    image = cv2.rectangle(img=image, 
                          pt1=(int(type_indent * width), 0),
                          pt2=(int(type_indent * width + doc_type_width * width), int(height)),
                          color=(0, 0, 255),
                          thickness=2)
    
    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    if index == 2:
        break

In [11]:
list_of_deltas = []

In [12]:
%%time
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_act_1/' + row['filename'])
    height, width, _ = image.shape
    
    x1 = int(row['doc_type_box_left'] * width)
    x2 = int((row['doc_type_box_left'] + row['doc_type_box_width']) * width)
    
    list_of_deltas.append(doc_type_region * width - x2)

CPU times: user 25 s, sys: 5.88 s, total: 30.9 s
Wall time: 31.1 s


In [13]:
# list_of_deltas

In [14]:
max(list_of_deltas)

285.3597504789895

In [15]:
min(list_of_deltas)

0.07553063807563376

In [16]:
pd.Series(list_of_deltas).describe()

count    200.000000
mean     164.875963
std       53.489420
min        0.075531
25%      136.133148
50%      176.229927
75%      200.280225
max      285.359750
dtype: float64

**Вывод:** регион для поиска типа документа имеет значения

отступ по x = 0.0163

ширина региона = 0.0914

### Расположение номера документа

In [17]:
delta_x_number_array = (data['doc_number_box_left'] - data['doc_type_box_width'] - data['doc_type_box_left'])

delta_x_number = delta_x_number_array.describe()['min']
delta_x_number

0.015574685613894204

In [18]:
delta_y_number_array = (data['doc_number_box_top'] - data['doc_type_box_height'] - data['doc_type_box_top'])

delta_y_number = delta_y_number_array.describe()['min'] 
delta_y_number

-0.012905092274742699

In [19]:
height_number = delta_y_number_array.describe()['max'] - delta_y_number_array.describe()['min'] + data['doc_number_box_height'].describe()['max']
height_number

0.019678520256959903

In [20]:
width_number = delta_x_number_array.describe()['max'] - delta_x_number_array.describe()['min'] + data['doc_number_box_width'].describe()['max']
width_number

0.09470749397412069

#### Визуализация результатов

In [22]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_act_1/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.2, fy=0.2)
    height, width, _ = image.shape
    
    x1 = int(row['doc_type_box_left'] * width)
    y1 = int(row['doc_type_box_top'] * height)
    x2 = int((row['doc_type_box_left'] + row['doc_type_box_width']) * width)
    y2 = int((row['doc_type_box_top'] + row['doc_type_box_height']) * height)
        
    delta_x_number_ = int(delta_x_number * width)
    delta_y_number_ = int(delta_y_number * height)
    width_number_ = int(width_number * width)
    height_number_ = int(height_number * width)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    image = cv2.rectangle(img=image,
                            pt1=(int(x2 + delta_x_number_), 
                                int(y2 + delta_y_number_)),
                            pt2=(int(x2 + delta_x_number_ + width_number_),
                                int(y2 + delta_y_number_ + height_number_)),
                            color=(0, 0, 255),
                            thickness=2)

    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    if index == 2:
        break

**Вывод:** принимаем следующие параметры региона **Номер документа**

дельта по x = 0.0156

дельта по y = -0.012905

ширина региона = 0.0947

высота региона = 0.0197

 ### Расположение даты документа

In [63]:
delta_x_date_array = (data['doc_date_box_left'] - data['doc_type_box_width'] - data['doc_type_box_left'])

delta_x_date = delta_x_date_array.describe()['min']
delta_x_date

0.102887895629479

In [64]:
delta_y_date_array = (data['doc_date_box_top'] - data['doc_type_box_height'] - data['doc_type_box_top'])

delta_y_date = delta_y_date_array.describe()['min']
delta_y_date

-0.0117829103378086

In [65]:
height_date = delta_y_date_array.describe()['max'] - delta_y_date_array.describe()['min'] + data['doc_date_box_height'].describe()['max']
height_date

0.017186463301020798

In [66]:
width_date = delta_x_date_array.describe()['max'] - delta_x_date_array.describe()['min'] + data['doc_date_box_width'].describe()['max']
width_date

0.11057689874111219

#### Визуализация результатов

In [67]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_act_1/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.2, fy=0.2)
    height, width, _ = image.shape
    
    x1 = int(row['doc_type_box_left'] * width)
    y1 = int(row['doc_type_box_top'] * height)
    x2 = int((row['doc_type_box_left'] + row['doc_type_box_width']) * width)
    y2 = int((row['doc_type_box_top'] + row['doc_type_box_height']) * height)
        
    delta_x_date_ = int(delta_x_date * width)
    delta_y_date_ = int(delta_y_date * height)
    width_date_ = int(width_date * width)
    height_date_ = int(height_date * width)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    image = cv2.rectangle(img=image,
                            pt1=(int(x2 + delta_x_date_), 
                                int(y2 + delta_y_date_)),
                            pt2=(int(x2 + delta_x_date_ + width_date_),
                                int(y2 + delta_y_date_ + height_date_)),
                            color=(0, 0, 255),
                            thickness=2)

    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    if index == 2:
        break

**Вывод:** принимаем следующие параметры региона **Дата документа**

дельта по x = 0.1028

дельта по y = -0.0118

ширина региона = 0.1106

высота региона = 0.0172

### Дефолтные значения для номера документа

In [68]:
doc_number_x1 = data['doc_number_box_left'].min()
doc_number_x1

0.0577101606365272

In [70]:
doc_number_y1 = data['doc_number_box_top'].min()
doc_number_y1

0.0124055631089079

In [69]:
doc_number_x2 = data['doc_number_box_left'].max() + data['doc_number_box_width'].max()
doc_number_x2

0.20151005190855098

In [71]:
doc_number_y2 = data['doc_number_box_top'].max() + data['doc_number_box_height'].max()
doc_number_y2

0.8381319073889654

In [72]:
doc_number_default_width = doc_number_x2 - doc_number_x1
doc_number_default_width

0.14379989127202378

In [73]:
doc_number_default_height = doc_number_y2 - doc_number_y1
doc_number_default_height

0.8257263442800574

#### Визуализация результатов

In [74]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_act_1/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.15, fy=0.15)
    height, width, _ = image.shape
        
    x1 = int(doc_number_x1 * width)
    y1 = int(doc_number_y1 * height)
    x2 = int(doc_number_x2 * width)
    y2 = int(doc_number_y2 * height)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    if index == 2:
        break

**Вывод:** принимаем следующие параметры дефолтного региона **Номер документа**

Левый верхний угол координаиа x = 0.0577

Левый верхний угол координата y = 0.0124

ширина региона = 0.1438

высота региона = 0.8258

### Дефолтные значения для даты документа

In [75]:
doc_date_x1 = data['doc_date_box_left'].min()
doc_date_x1

0.1523474159694511

In [76]:
doc_date_y1 = data['doc_date_box_top'].min()
doc_date_y1

0.0121578700268369

In [77]:
doc_date_x2 = data['doc_date_box_left'].max() + data['doc_date_box_width'].max()
doc_date_x2

0.2850617560174828

In [78]:
doc_date_y2 = data['doc_date_box_top'].max() + data['doc_date_box_height'].max()
doc_date_y2

0.8370626842492491

In [79]:
doc_date_default_width = doc_date_x2 - doc_date_x1
doc_date_default_width

0.1327143400480317

In [80]:
doc_date_default_height = doc_date_y2 - doc_date_y1
doc_date_default_height

0.8249048142224122

#### Визуализация результатов

In [81]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_act_1/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.15, fy=0.15)
    height, width, _ = image.shape
        
    x1 = int(doc_date_x1 * width)
    y1 = int(doc_date_y1 * height)
    x2 = int(doc_date_x2 * width)
    y2 = int(doc_date_y2 * height)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    if index == 2:
        break

**Вывод:** принимаем следующие параметры дефолтного региона **Дата документа**

Левый верхний угол координаиа x = 0.1524

Левый верхний угол координата y = 0.0122

ширина региона = 0.1328

высота региона = 0.8249